<a href="https://colab.research.google.com/github/OMatheusWander/BrasilAPI/blob/master/AULA_DE_STATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

forvalues i = 5/11 {
 cls
 clear all
 import delimited C:\Users\06592896957\Desktop\PNAD_COVID_`i'.csv, clear
 keep if uf == 42
 keep if v1022 == 1
 keep if v1023 == 1
 save C:\Users\06592896957\Desktop\PNAD_COVID_`i'.dta, replace
}

cls
clear all
use C:\Users\Usuario\Seafile\Seafile\stuffs\exercises\ler_microdados\PNAD_COVID_5.dta
forvalues i = 6/11 {
 append using C:\Users\Usuario\Seafile\Seafile\stuffs\exercises\ler_microdados\PNAD_COVID_`i'.dta
 save C:\Users\Usuario\Seafile\Seafile\stuffs\exercises\ler_microdados\data_COVID.dta, replace
}




















*note que são só entrevistas em Floripa
tab rm_ride capital
*note o número de entrevistados por mês e semana
tab v1012 v1013
export excel using "C:\Users\Usuario\Seafile\Seafile\stuffs\exercises\ler_microdados\data_COVID.xlsx", firstrow(variables) replace


cls
clear all
use C:\Users\Usuario\Seafile\Seafile\stuffs\exercises\ler_microdados\data_COVID.dta

egen id = group(v1008 estrato upa)
rename v1013 mes

keep if a001a == 1

sort id mes
bysort id: gen times = _N

tab times

drop if times == 1

rename a002 idade

gen homem = cond(a003 == 1, 1, 0)

*tw (kdensity idade if homem == 0) (kdensity idade if homem == 1)

gen branco = cond(a004 == 1, 1, 0)

rename a005 edu

tab edu, m

rename c008 horas
rename c01012 w

order id mes idade homem branco edu horas w

drop ano-times

gen lnw = ln(w)

sort id mes

tsset id mes
tsfill, full

bysort id: gen ok = cond(_n == 1 & idade != . & homem != . & branco != . & edu!=1,1,0)
bysort id: replace ok = ok[_n-1] if _n > 1
drop if ok == 0
drop ok

bysort id: egen idade2 = max(idade)
drop if idade2 == .
bysort id: gen ok = cond(idade == . | idade == idade2 | idade == idade2-1 ,1,0)
bysort id: egen ok2 = mean(ok)
drop if ok2 != 1
drop ok ok2 idade2

bysort id: replace idade = idade[_n-1] if idade == .

bysort id: replace homem = homem[_n-1] if homem == .
bysort id: egen homem2 = mean(homem)
drop if homem != homem2
drop homem2

bysort id: replace branco = branco[_n-1] if branco == .
bysort id: egen branco2 = mean(branco)
drop if branco != branco2
drop branco2

bysort id: egen edu2 = max(edu)
drop if edu2 == .
bysort id: gen ok = cond(edu == . | edu == edu2 | edu == edu2-1 ,1,0)
bysort id: egen ok2 = mean(ok)
drop if ok2 != 1
drop ok ok2 edu2

sort id mes

tsset id mes
tsfill, full

xtset id mes
gen idade2 = idade^2
xtreg lnw idade idade2 i.edu, fe

